# <center><i><code style="background:lightblue;color:black">Forecasting on Avalability of Hospital Beds </code> </i></center>

##### Importing necessary Libraries

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import seaborn as sns
from pandas.plotting import lag_plot

from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from pandas.plotting import autocorrelation_plot


import statsmodels.graphics.tsaplots as tsa_plot

from ml_metrics import rmse
from sklearn.metrics import mean_squared_error
from math import sqrt

from statsmodels.tsa.ar_model import AutoReg
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.holtwinters import SimpleExpSmoothing
from statsmodels.tsa.holtwinters import Holt
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from pmdarima import auto_arima
import fbprophet 
import pickle


import warnings
warnings.filterwarnings('ignore')

ModuleNotFoundError: No module named 'pmdarima'

##### Importing the data and having a glance at it


In [ ]:
data=pd.read_csv('Beds_Occupied.csv')
data.head()

### <center><code style="background:lightblue;color:black">Exploratory Data Analysis(EDA)</code>

##### Looking over the datatype and null values if present any in our dataset

In [ ]:
data.info()

##### Renaming the column names in order to avoid spacing

In [ ]:
data.columns=['CD','TIB']

##### Changing the collection date column into datetime format

In [ ]:
data['CD']=pd.to_datetime(data['CD'],format='%d-%m-%Y')

##### Looking over the datatype and null values if present any in our dataset

In [ ]:
data.info()

##### Identifying the Missing values in a year from our data

In [ ]:
range_dates = pd.date_range(start=data.CD.min(), end=data.CD.max())
range_dates

##### setting our index column to CD column(datetime)

In [ ]:
data=data.set_index('CD').reindex(range_dates).rename_axis('CD').reset_index()

##### Checking the rows and columns

In [ ]:
data.shape

##### Listing the missing values from our data

In [ ]:
data.isna().sum()

##### Filling the missing TIB dates with the median value 

In [ ]:
data=data.fillna(data.TIB.mean())

##### looking for missing values 

In [ ]:
data.isna().sum()

##### Creating the target variable for further evaluation

In [ ]:
data['AIB']=900-data['TIB']
data.head()

##### Looking into the statistical detail of the data

In [ ]:
data.describe()

In [ ]:
data=data.set_index('CD')
data

##### Dropping the TIB column and considering only the target variable (Avail_beds)

In [ ]:
data=data.drop('TIB',axis=1)

### <center><code style="background:lightblue;color:black">Visualizing the data with different plots</code>

##### Plotting the line plot to check the trend in data

In [ ]:
plt.figure(figsize=(10,5))
data.AIB.plot(linewidth=2)
plt.title('Line Plot')
plt.xlabel('Months')
plt.ylabel('Available Beds')

>From the line plot above it is evident that our data has no upward or downward trend

#####  Plotting the KDE plot to check the data is  distributed normally or not

In [ ]:
plt.figure(figsize=(10,5))
data.AIB.plot(kind='kde')
plt.title('KDE Plot')
plt.xlabel('Availabile Beds')


>From the above plot it depicts that the data follows almost normal distribution

##### Plotting the Histogram plot to check distribution of frequency on the data

In [ ]:
plt.figure(figsize=(15,7))
data.AIB.plot(kind='hist')
plt.title('Histogram')
plt.xlabel('Available Beds')


>From the histrogram plot above it is observed that the data is slightly skewed towards left

##### Creating a month column, to create a box plot of availability of beds for each month

In [ ]:
data['month']=data.index.month


In [ ]:
plt.rcParams['figure.figsize']=15,10
box = data.boxplot(by='month',column=['AIB'],patch_artist=True,grid=False)
plt.ylabel('Available beds')
plt.show()

>From the box plot above it is observed that outliers are present in the data

##### Plotting the lag plot to check the randomness in the data

In [ ]:

plt.rcParams['figure.figsize']=15,7
plt.title('Lag Plot')
lag_plot(data.AIB)
plt.xlabel('available beds')


>From the lagplot above it is evident that our data has randomness since it exhibits non identifiable patterns in it

### <center><code style="background:lightblue;color:black">Tests to check the stationarity</code>

> <b>1)Rolling Statistics</b><br>
 > <b>2)Dicky Fuller Test</b>

##### Determining the Rolling Statistics to check the Stationarity of the data

In [ ]:
#Providing the window of 12days and taking average of it
rolling_mean=data.AIB.rolling(window=7).mean()
rolling_mean

In [ ]:
#Providing the window for 12days and taking standard deviation of it
rolling_std=data.AIB.rolling(window=7).std()
rolling_std

##### Visualizing the Rolling Statistics for Mean and Standard Deviation

In [ ]:
plt.figure(figsize=(15,7))
plt.plot(data.AIB,label='Orginal AIB')
plt.plot(rolling_mean,label="Mean of AIB")
plt.plot(rolling_std,label="STd of AIB")
plt.legend(loc='best')

plt.title('Rolling Statistics')
plt.xlabel('Months')
plt.ylabel('Available Beds')

##### Determining the Dicky Fuller test to check the Stationarity of the data

In [ ]:
hyp_test=adfuller(data['AIB'],autolag='AIC')
output=pd.Series(hyp_test[0:4],index=['test stats','p-value','lags used','no. of obs used'])

for key, value in hyp_test[4].items():
    output['critical value (%s)'%key]=value
print(output)

##### Looking into the Decompositional plots

In [ ]:
decompose=seasonal_decompose(data.AIB,period=7)
decompose.plot()
plt.show()

In [ ]:
autocorrelation_plot(data.AIB)

In [ ]:
tsa_plot.plot_acf(data.AIB,lags=12)
plt.xlabel('Lags')
plt.ylabel('Autocorrelation')
plt.show()

In [ ]:

tsa_plot.plot_pacf(data.AIB,lags=12)
plt.xlabel('Lags')
plt.ylabel('PartialAutocorrelation')
plt.show()

### <center><code style="background:lightblue;color:black">Model Building</code>

##### Splitting the data into train and test data

In [ ]:
#train is 90% of total data and test is 10% of total data
train=data.head(336)
test=data.tail(30)

##### Dropping the month column for both train and test data

In [ ]:
train=train.drop(['month'],axis=1)
test=test.drop(['month'],axis=1)
train

### Auto regressive model 

In [ ]:
model1=AutoReg(train['AIB'],lags=1).fit()

In [ ]:
pred1_train=model1.predict(0,len(train)-1)
pred1_test=model1.predict(len(train),len(data)-1)
pred1_data=model1.predict(0,len(data)-1)

In [ ]:
rmse1_train=rmse(pred1_train[1:],train.AIB[1:])
rmse1_test=rmse(pred1_test,test.AIB)
rmse1_data=rmse(pred1_data[1:],data.AIB[1:])
rmse1_train,rmse1_test,rmse1_data

In [ ]:
plt.figure(figsize=(15,5))
plt.plot(train.AIB,label='train')
plt.plot(test.AIB,label='actual')
plt.plot(pred1_train,label='train_prediction')
plt.plot(pred1_test,label='test_prediction')
plt.legend(loc='best')
plt.title('Autoregression')
plt.show()



### Moving Average Model

In [ ]:
model2=ARIMA(train.AIB,order=(0,0,1)).fit()

In [ ]:
pred2_test=model2.predict(test.index[0],test.index[-1])
pred2_train=model2.predict(train.index[0],train.index[-1])
pred2_data=model2.predict(0,len(data)-1)

In [ ]:
rmse2_test=rmse(pred2_test,test.AIB)
rmse2_train=rmse(pred2_train,train.AIB)
rmse2_data=rmse(pred2_data,data.AIB)
rmse2_train,rmse2_test,rmse2_data

In [ ]:
plt.figure(figsize=(15,5))
plt.plot(train.AIB,label='train')
plt.plot(test.AIB,label='actual')
plt.plot(pred2_train,label='train_prediction')
plt.plot(pred2_test,label='test_prediction')
plt.legend(loc='best')
plt.show()



###  AutoRegressive Moving Average(ARMA)

In [ ]:
model3=ARIMA(train.AIB,order=(3,0,6)).fit()

In [ ]:
pred3_test=model3.predict(len(train),len(data)-1)
pred3_train=model3.predict(train.index[0],train.index[-1])
pred3_data=model3.predict(0,len(data)-1)
rmse3_test=rmse(pred3_test,test.AIB)
rmse3_train=rmse(pred3_train,train.AIB)
rmse3_data=rmse(pred3_data,data.AIB)
rmse3_train,rmse3_test,rmse3_data

### forecast

In [ ]:
model3_data=ARIMA(data.AIB,order=(3,0,6)).fit()
model3_forecast=model3_data.forecast(30)

In [ ]:
model3_forecast

In [ ]:
plt.figure(figsize=(15,5))
plt.plot(train.AIB,label='train')
plt.plot(test.AIB,label='actual')
plt.plot(pred3_train,label='train_prediction',linewidth=2)
plt.plot(pred3_test,label='test_prediction',linewidth=2)
plt.plot(model3_forecast,label='forecast',linewidth=2)
plt.legend(loc='best')
plt.title('ARMA model')
plt.xlabel('Months')
plt.ylabel('Available Beds')
plt.show()



### AutoRegressive Integrated Moving Average(ARIMA)

In [ ]:
model4=ARIMA(train.AIB,order=(1,1,1)).fit()

In [ ]:
pred4_test=model4.predict(len(train),len(data)-1)
pred4_train=model4.predict(train.index[0],train.index[-1])
pred4_data=model4.predict(train.index[0],data.index[-1])
rmse4_test=rmse(pred4_test,test.AIB)
rmse4_train=rmse(pred4_train,train.AIB)
rmse4_data=rmse(pred4_data,data.AIB)
rmse4_train,rmse4_test,rmse4_data

In [ ]:
plt.figure(figsize=(15,5))
plt.plot(train.AIB[1:],label='train')
plt.plot(test.AIB,label='actual')
plt.plot(pred4_train[1:],label='train_prediction')
plt.plot(pred4_test,label='test_prediction')
plt.legend(loc='best')
plt.show()



### Seasonal ARIMA(SARIMA )

In [ ]:
model5=SARIMAX(train.AIB,order=(1,0,1),seasonal_order=(1,0,1,7)).fit()

In [ ]:
pred5_test=model5.predict(len(train),len(data)-1)
pred5_train=model5.predict(train.index[0],train.index[-1])
pred5_data=model5.predict(data.index[0],len(data)-1)
rmse5_test=rmse(pred5_test,test.AIB)
rmse5_train=rmse(pred5_train,train.AIB)
rmse5_data=rmse(pred5_data,data.AIB)
rmse5_train,rmse5_test,rmse5_data

In [ ]:
plt.figure(figsize=(15,5))
plt.plot(train.AIB[1:],label='train')
plt.plot(test.AIB,label='actual')
plt.plot(pred5_train[1:],label='train_prediction')
plt.plot(pred5_test,label='test_prediction')
plt.legend(loc='best')
plt.show()




### Simple Exponential Smoothing 

In [ ]:
model7=SimpleExpSmoothing(train.AIB).fit(smoothing_level=0.2)
pred7_test=model7.predict(len(train),len(data)-1)
pred7_train=model7.predict(train.index[0],train.index[-1])
pred7_data=model7.predict(data.index[0],data.index[-1])
rmse7_test=rmse(pred7_test,test.AIB)
rmse7_train=rmse(pred7_train,train.AIB)
rmse7_data=rmse(pred7_data,data.AIB)
rmse7_train,rmse7_test,rmse7_data

In [ ]:
plt.figure(figsize=(15,5))
plt.plot(train.AIB,label='train')
plt.plot(test.AIB,label='actual')
plt.plot(pred7_train,label='train_prediction')
plt.plot(pred7_test,label='test_prediction')
plt.legend(loc='best')
plt.show()



### Holts Trend Method

In [ ]:
model8=Holt(train.AIB).fit(smoothing_level=0.1)
pred8_test=model8.predict(len(train),len(data)-1)
pred8_train=model8.predict(train.index[0],train.index[-1])
pred8_data=model8.predict(data.index[0],data.index[-1])
rmse8_test=rmse(pred8_test,test.AIB)
rmse8_train=rmse(pred8_train,train.AIB)
rmse8_data=rmse(pred8_data,data.AIB)
rmse8_train,rmse8_test,rmse8_data

In [ ]:
plt.figure(figsize=(15,5))
plt.plot(train.AIB,label='train')
plt.plot(test.AIB,label='actual')
plt.plot(pred8_train,label='train_prediction')
plt.plot(pred8_test,label='test_prediction')
plt.legend(loc='best')
plt.show()




### Holts Winter Exponential Smoothing(HWES)

In [ ]:
model9=ExponentialSmoothing(train.AIB).fit()
pred9_test=model9.predict(len(train),len(data)-1)
pred9_train=model9.predict(train.index[0],train.index[-1])
pred9_data=model9.predict(data.index[0],data.index[-1])
rmse9_test=rmse(pred9_test,test.AIB)
rmse9_train=rmse(pred9_train,train.AIB)
rmse9_data=rmse(pred9_data,data.AIB)
rmse9_train,rmse9_test,rmse9_data

In [ ]:
plt.figure(figsize=(15,5))
plt.plot(train.AIB,label='train')
plt.plot(test.AIB,label='actual')
plt.plot(pred9_train,label='train_prediction')
plt.plot(pred9_test,label='test_prediction')
plt.legend(loc='best')
plt.show()




### Holts Winter Exponential Smoothing with Additive Seasonality and Additive Trend

In [ ]:
model10=ExponentialSmoothing(train.AIB,seasonal="add",seasonal_periods=5).fit()
pred10_test=model10.predict(test.index[0],test.index[-1])
pred10_train=model10.predict(train.index[0],train.index[-1])
pred10_data=model10.predict(data.index[0],data.index[-1])
rmse10_test=rmse(pred10_test,test.AIB)
rmse10_train=rmse(pred10_train,train.AIB)
rmse10_data=rmse(pred10_data,data.AIB)
rmse10_train,rmse10_test,rmse10_data

In [ ]:
plt.figure(figsize=(15,5))
plt.plot(train.AIB,label='train')
plt.plot(test.AIB,label='actual')
plt.plot(pred10_train,label='train_prediction')
plt.plot(pred10_test,label='test_prediction')
plt.legend(loc='best')
plt.show()





### HWES with Multiplicative Seasonality and Additive Trend

In [ ]:
model11=ExponentialSmoothing(train.AIB,seasonal="mul",seasonal_periods=8).fit()
pred11_test=model11.predict(test.index[0],test.index[-1])
pred11_train=model11.predict(train.index[0],train.index[-1])
pred11_data=model11.predict(data.index[0],data.index[-1])
rmse11_test=rmse(pred11_test,test.AIB)
rmse11_train=rmse(pred11_train,train.AIB)
rmse11_data=rmse(pred11_data,data.AIB)
rmse11_train,rmse11_test,rmse11_data

### forecast

In [ ]:
model11_data=ExponentialSmoothing(data.AIB,seasonal="mul",seasonal_periods=8).fit()

In [ ]:
model11_forecast=model11_data.forecast(30)


In [ ]:
plt.figure(figsize=(15,5))
plt.plot(train.AIB,label='train')
plt.plot(test.AIB,label='actual')
plt.plot(pred11_train,label='train_prediction')
plt.plot(pred11_test,label='test_prediction')
plt.plot(model11_forecast,label="Forecast")
plt.legend(loc='best')
plt.show()

### Prophet

In [ ]:
df=data
df=df.drop(['month'],axis=1)
df

In [ ]:
df=df.reset_index()
df.columns=['ds','y']
df

In [ ]:
df_train=df[:336]
df_test=df[30:]


In [ ]:
model_prophet=fbprophet.Prophet(yearly_seasonality = False,daily_seasonality=True).fit(df_train)

In [ ]:
pred_prophet_test=model_prophet.predict(df_test)
pred_prophet_train=model_prophet.predict(df_train)
pred_prophet_data=model_prophet.predict(df)

In [ ]:
pred_prophet_test.head()

In [ ]:
rmse_prophet_test=rmse(df_test.y,pred_prophet_test.yhat)
rmse_prophet_train=rmse(df_train.y,pred_prophet_train.yhat)
rmse_prophet_data=rmse(df.y,pred_prophet_data.yhat)
rmse_prophet_train,rmse_prophet_test,rmse_prophet_data

In [ ]:
plt.plot(df.ds,df.y,label='original')
plt.plot(pred_prophet_test.ds,pred_prophet_test.yhat,label='test predicted',linewidth=2)
plt.plot(pred_prophet_train.ds,pred_prophet_train.yhat,label='train predicted',linewidth=2)
plt.legend(loc='best')

In [ ]:
model_prophet.plot(pred_prophet_test)
model_prophet.plot(pred_prophet_train)
plt.show()

##### Forecasting for the Prophet

In [ ]:
model_prophet_forecast=fbprophet.Prophet(yearly_seasonality = False,daily_seasonality=True).fit(df)

In [ ]:
future = model_prophet_forecast.make_future_dataframe(periods=30)

In [ ]:
future.tail()

In [ ]:
prophet_forecast = model_prophet.predict(future)
prophet_forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail(30)

In [ ]:
fig1 = model_prophet_forecast.plot(prophet_forecast)

In [ ]:
fig2 = model_prophet_forecast.plot_components(prophet_forecast)

##### Tabulating the RMSE score for all the models built

In [ ]:
t={"model":pd.Series(["AR","MA","ARMA","ARIMA","SARIMA","SES","Holts trend","HWES","hwes additive seasonality & additive trend","hews multiplicative seasonality & additive trend","auto arima ","prophet"]),
  "RMSE_test":[rmse1_test,rmse2_test,rmse3_test,rmse4_test,rmse5_test,rmse7_test,rmse8_test,rmse9_test,rmse10_test,rmse11_test,rmse_try_test,rmse_prophet_test],
   "RMSE_train":[rmse1_train,rmse2_train,rmse3_train,rmse4_train,rmse5_train,rmse7_train,rmse8_train,rmse9_train,rmse10_train,rmse11_train,rmse_try_train,rmse_prophet_train],
   "RMSE_full_data":[rmse1_data,rmse2_data,rmse3_data,rmse4_data,rmse5_data,rmse7_data,rmse8_data,rmse9_data,rmse10_data,rmse11_data,rmse_try_data,rmse_prophet_data]
  }
t=pd.DataFrame(t)
t.style.highlight_min(color = 'lightgreen', axis = 0) 

In [ ]:
pickle.dump(model_prophet_forecast,open('prophet.pkl','wb'))